# **Zindi Air Quality Prediction from Low-Cost IoT devices Contest**


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import optuna
import os
import warnings
warnings.filterwarnings('ignore')

## **Step 1: Load and Explore Data**


In [2]:
print("Loading data...")
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

train_data.head()

Loading data...


,ID,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,device_name,CO2
0,ID_000001,28.975,74.475,2480.0,3476.5,1572.0,1997.0,alpha,585.75
1,ID_000002,31.900,66.500,3813.0,2726.0,4145.0,3180.0,alpha,613.00
2,ID_000003,31.675,60.015,2811.0,1563.5,4250.0,2708.5,alpha,616.50
3,ID_000004,31.580,59.220,2844.0,1597.0,4310.0,2723.0,alpha,642.50
4,ID_000005,31.690,62.030,3159.5,1120.5,5519.5,1219.0,alpha,622.00


In [3]:
# Store IDs before feature engineering
train_ids = train_data['ID'].copy()
test_ids = test_data['ID'].copy()

## **Step 2: Handle missing data**

In [4]:
# Handle important IDs
required_ids = ['ID_007308', 'ID_007315', 'ID_007323', 'ID_007324', 'ID_007326']
missing_ids = [id_value for id_value in required_ids if id_value not in test_data['ID'].values]

if missing_ids:
    new_rows = pd.DataFrame({'ID': missing_ids})
    test_data = pd.concat([test_data, new_rows], ignore_index=True)
    print(f"Added missing IDs: {missing_ids}")

## **Step 3: Feature Engineering**



In [5]:
def create_advanced_features(df):
    
    df = df.copy()
    sensor_cols = ['MQ7_analog', 'MQ9_analog', 'MG811_analog', 'MQ135_analog']
    
    # Basic sensor features
    df['Sensor_mean'] = df[sensor_cols].mean(axis=1)
    df['Sensor_std'] = df[sensor_cols].std(axis=1)
    df['Sensor_median'] = df[sensor_cols].median(axis=1)
    df['Sensor_max'] = df[sensor_cols].max(axis=1)
    df['Sensor_min'] = df[sensor_cols].min(axis=1)
    df['Sensor_range'] = df['Sensor_max'] - df['Sensor_min']
    
    # statistical features
    df['Sensor_skew'] = df[sensor_cols].skew(axis=1)
    df['Sensor_kurtosis'] = df[sensor_cols].kurtosis(axis=1)
    
    # Ratio features
    for i in range(len(sensor_cols)):
        for j in range(i+1, len(sensor_cols)):
            ratio_name = f'ratio_{sensor_cols[i]}_{sensor_cols[j]}'
            df[ratio_name] = df[sensor_cols[i]] / (df[sensor_cols[j]] + 1e-6)
    
    # Temperature Compensation with advanced scaling
    temp_ref = 25.0
    humidity_ref = 50.0
    for col in sensor_cols:
        # Temperature compensation
        temp_factor = 1 + 0.02 * (df['Temperature'] - temp_ref)
        # Humidity compensation
        humid_factor = 1 + 0.01 * (df['Humidity'] - humidity_ref)
        
        df[f'{col}_temp_comp'] = df[col] * temp_factor
        df[f'{col}_humid_comp'] = df[col] * humid_factor
        df[f'{col}_full_comp'] = df[col] * temp_factor * humid_factor
    
    # Environmental Features
    df['Temp_Humid_interaction'] = df['Temperature'] * df['Humidity']
    df['Temp_Humid_ratio'] = df['Temperature'] / (df['Humidity'] + 1e-6)
    df['Temp_squared'] = df['Temperature'] ** 2
    df['Humid_squared'] = df['Humidity'] ** 2
    
    return df

## **Step 4: Apply Feature Engineering to Data**

In [6]:
print("Creating advanced features...")
train_features = create_advanced_features(train_data.drop(['ID', 'device_name', 'CO2'], axis=1))
test_features = create_advanced_features(test_data.drop(['ID', 'device_name'], axis=1))
numeric_cols = train_features.select_dtypes(include=['float64', 'int64']).columns

Creating advanced features...


## **Step 5: Add Polynomial Features**


In [7]:
print("Creating polynomial features...")
poly = PolynomialFeatures(degree=2, include_bias=False)

poly_features_train = poly.fit_transform(train_features[numeric_cols])
poly_features_test = poly.transform(test_features[numeric_cols])

feature_names = (
    list(numeric_cols) + 
    [f"poly_{i}" for i in range(poly_features_train.shape[1] - len(numeric_cols))]
)
# Scale features
print("Scaling features...")
scaler = RobustScaler()
X_scaled = scaler.fit_transform(poly_features_train)
test_scaled = scaler.transform(poly_features_test)
y = train_data['CO2'].values

Creating polynomial features...
Scaling features...


## **Step 6: Hyperparameter Optimization with Optuna**

In [8]:
def objective(trial, X, y, cv):
    """Optuna objective function for XGBoost optimization"""
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'tree_method': 'hist',
        'random_state': 42
    }
    
    scores = []
    for train_idx, valid_idx in cv.split(X):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        
        model = XGBRegressor(**param)
        model.fit(X_train, y_train,
                 eval_set=[(X_valid, y_valid)],
                 verbose=False)
        
        pred = model.predict(X_valid)
        rmse = np.sqrt(mean_squared_error(y_valid, pred))
        scores.append(rmse)
    
    return np.mean(scores)

## **Step 7: Run Hyperparameter Optimization**

In [ ]:
# Hyperparameter optimization
print("Optimizing hyperparameters...")
cv = KFold(n_splits=5, shuffle=True, random_state=42)

study = optuna.create_study(direction='minimize')
study.optimize(
    lambda trial: objective(trial, X_scaled, y, cv),
    n_trials=50,
    timeout=3600  # 1 hour timeout
)

best_params = study.best_params
best_params['tree_method'] = 'hist'
best_params['random_state'] = 42
print("Best parameters:", best_params)

## **Step 8: Train Final Model**

In [17]:
# Best hyperparameters from Optuna
best_params = {
    'max_depth': 5,
    'learning_rate': 0.042000323432903566,
    'n_estimators': 1128,
    'min_child_weight': 3,
    'subsample': 0.8388792014281559,
    'colsample_bytree': 0.6496286198943079,
    'gamma': 1.5039649063232274,
    'reg_alpha': 4.097232713617798,
    'reg_lambda': 2.3317882013391,
    'max_bin': 318,
    'grow_policy': 'depthwise',
    'tree_method': 'hist',
    'random_state': 42
}

print("Training final model...")
final_model = XGBRegressor(**best_params)
final_model.fit(X_scaled, y)

print("Making predictions...")
predictions = final_model.predict(test_scaled)

submission = pd.DataFrame({
    'ID': test_ids,
    'CO2': predictions
})

mean_pred = predictions.mean()
for id_value in required_ids:
    if id_value not in submission['ID'].values:
        submission = pd.concat([
            submission,
            pd.DataFrame({'ID': [id_value], 'CO2': [mean_pred]})
        ])

submission = submission.sort_values('ID').reset_index(drop=True)

os.makedirs('outputs', exist_ok=True)
submission_path = os.path.join('outputs', 'submission.csv')
submission.to_csv(submission_path, index=False)

print("Training completed! Best RMSE: 5.7107")
print(f"Submission saved with {len(submission)} entries")

Training final model...
Making predictions...
Training completed! Best RMSE: 5.7107
Submission saved with 1292 entries


## **Step 9: Feature Importance Analysis**


In [18]:
# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Log top 10 important features
print("\nTop 10 Important Features:")
print(feature_importance.head(10))


Top 10 Important Features:
      feature  importance
363  poly_327    0.046091
447  poly_411    0.042238
464  poly_428    0.038274
459  poly_423    0.031643
115   poly_79    0.027604
337  poly_301    0.023573
466  poly_430    0.022867
628  poly_592    0.016849
465  poly_429    0.015622
561  poly_525    0.013871
